In [86]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [87]:
# url that we are scraping
url = "https://www.basketball-reference.com/draft/NBA_2017.html"

# this is the html from the given url
html = urlopen(url)
html = html.read()
print(html)

b'\n<!DOCTYPE html>\n<html data-version="klecko-" data-root="/home/bbr/build" itemscope itemtype="https://schema.org/WebSite" lang="en" class="no-js" >\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n    <link rel="dns-prefetch" href="https://d2p3bygnnzw9w3.cloudfront.net/req/201803121" />\n\n<!-- no:cookie fast load the css.           -->\n<link rel="preconnect" href="https://d2p3bygnnzw9w3.cloudfront.net" crossorigin>\n<link rel="preconnect" href="https://d2cwpp38twqe55.cloudfront.net" crossorigin>\n <style>html,body{margin:0;padding:0;font:14px/1.25 "Helvetica Neue",helvetica,arial,sans-serif;color:#000}body{position:relative;background:#c9cbcd;z-index:0;-webkit-text-size-adjust:none;-moz-text-size-adjust:none;-ms-text-size-adjust:none}a img{border:0}ul,li,ol{margin:0;padding:0;list-style-type:none}table th,table td{border:0}iframe{max-widt

In [88]:
soup = BeautifulSoup(html)

D:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file D:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [89]:
column_headers = ['Pk',
 'Tm',
 'Player',
 'College',
 'Yrs',
 'G',
 'MP',
 'PTS',
 'TRB',
 'AST',
 'FG%',
 '3P%',
 'FT%',
 'MP',
 'PTS',
 'TRB',
 'AST',
 'WS',
 'WS/48',
 'BPM',
 'VORP']

In [90]:
data_rows = soup.findAll('tr')[2:]  # skip the first 2 header rows

In [91]:
player_data = [[td.getText() for td in data_rows[i].findAll('td')]
            for i in range(len(data_rows))]

In [92]:
df = pd.DataFrame(player_data, columns=column_headers)

In [93]:
df.head()

,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,AST,...,3P%,FT%,MP,PTS,TRB,AST,WS,WS/48,BPM,VORP
0,1,PHI,Markelle Fultz,University of Washington,1,4,76,24,9,7,...,,.500,19.0,6.0,2.3,1.8,-0.1,-.087,-10.1,-0.2
1,2,LAL,Lonzo Ball,"University of California, Los Angeles",1,47,1589,483,326,333,...,.316,.459,33.8,10.3,6.9,7.1,1.7,.053,1.7,1.5
2,3,BOS,Jayson Tatum,Duke University,1,69,2102,919,346,102,...,.423,.823,30.5,13.3,5.0,1.5,5.9,.135,0.6,1.4
3,4,PHO,Josh Jackson,University of Kansas,1,67,1604,786,293,88,...,.266,.626,23.9,11.7,4.4,1.3,-0.7,-.022,-4.8,-1.1
4,5,SAC,De'Aaron Fox,University of Kentucky,1,61,1673,701,154,264,...,.331,.727,27.4,11.5,2.5,4.3,-0.5,-.014,-4.5,-1.1


In [94]:
# finding rows with missing data
df[df['Pk'].isnull()]

,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,AST,...,3P%,FT%,MP,PTS,TRB,AST,WS,WS/48,BPM,VORP
30,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
31,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [95]:
# removing rows with missing data
df = df[df.Player.notnull()]

In [96]:
# now all rows with missing data are gone
df[df['Pk'].isnull()]

,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,AST,...,3P%,FT%,MP,PTS,TRB,AST,WS,WS/48,BPM,VORP


In [97]:
# renaming columns - WS48 -> WS_per_48
df.rename(columns={'WS/48': 'WS_per_48'}, inplace=True)

In [98]:
# rename columns with '%' to '_Perc'
df.columns = df.columns.str.replace('%', '_Perc')

In [99]:
# also need to differentiate between per game stats and total cumulative stats
df.columns.values[13:17] = [df.columns.values[13:17][col] + 
                                  "_per_G" for col in range(4)]

In [100]:
print(df.columns)

Index(['Pk', 'Tm', 'Player', 'College', 'Yrs', 'G', 'MP', 'PTS', 'TRB', 'AST',
       'FG_Perc', '3P_Perc', 'FT_Perc', 'MP_per_G', 'PTS_per_G', 'TRB_per_G',
       'AST_per_G', 'WS', 'WS_per_48', 'BPM', 'VORP'],
      dtype='object')


In [101]:
df.dtypes # take a look at data types in each column

Pk           object
Tm           object
Player       object
College      object
Yrs          object
G            object
MP           object
PTS          object
TRB          object
AST          object
FG_Perc      object
3P_Perc      object
FT_Perc      object
MP_per_G     object
PTS_per_G    object
TRB_per_G    object
AST_per_G    object
WS           object
WS_per_48    object
BPM          object
VORP         object
dtype: object

In [102]:
# changing data types to proper data types
df = df.convert_objects(convert_numeric=True)
df.dtypes

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


Pk             int64
Tm            object
Player        object
College       object
Yrs          float64
G            float64
MP           float64
PTS          float64
TRB          float64
AST          float64
FG_Perc      float64
3P_Perc      float64
FT_Perc      float64
MP_per_G     float64
PTS_per_G    float64
TRB_per_G    float64
AST_per_G    float64
WS           float64
WS_per_48    float64
BPM          float64
VORP         float64
dtype: object

In [103]:
# dealing with null values - players who haven't played in NBA have null for their stats
df = df[:].fillna(0) # index all the columns and fill in the 0's

In [104]:
df.loc[:, 'Yrs':'AST'] = df.loc[:, 'Yrs': 'AST'].astype(int)

df.head()

,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,AST,...,3P_Perc,FT_Perc,MP_per_G,PTS_per_G,TRB_per_G,AST_per_G,WS,WS_per_48,BPM,VORP
0,1,PHI,Markelle Fultz,University of Washington,1,4,76,24,9,7,...,0.000,0.500,19.0,6.0,2.3,1.8,-0.1,-0.087,-10.1,-0.2
1,2,LAL,Lonzo Ball,"University of California, Los Angeles",1,47,1589,483,326,333,...,0.316,0.459,33.8,10.3,6.9,7.1,1.7,0.053,1.7,1.5
2,3,BOS,Jayson Tatum,Duke University,1,69,2102,919,346,102,...,0.423,0.823,30.5,13.3,5.0,1.5,5.9,0.135,0.6,1.4
3,4,PHO,Josh Jackson,University of Kansas,1,67,1604,786,293,88,...,0.266,0.626,23.9,11.7,4.4,1.3,-0.7,-0.022,-4.8,-1.1
4,5,SAC,De'Aaron Fox,University of Kentucky,1,61,1673,701,154,264,...,0.331,0.727,27.4,11.5,2.5,4.3,-0.5,-0.014,-4.5,-1.1


In [105]:
# adding column for draft year 
df.insert(0, 'Draft_Yr', 2017)
df.columns

Index(['Draft_Yr', 'Pk', 'Tm', 'Player', 'College', 'Yrs', 'G', 'MP', 'PTS',
       'TRB', 'AST', 'FG_Perc', '3P_Perc', 'FT_Perc', 'MP_per_G', 'PTS_per_G',
       'TRB_per_G', 'AST_per_G', 'WS', 'WS_per_48', 'BPM', 'VORP'],
      dtype='object')

In [119]:
df.dtypes

Draft_Yr       int64
Pk             int64
Tm            object
Player        object
College       object
Yrs            int32
G              int32
MP             int32
PTS            int32
TRB            int32
AST            int32
FG_Perc      float64
3P_Perc      float64
FT_Perc      float64
MP_per_G     float64
PTS_per_G    float64
TRB_per_G    float64
AST_per_G    float64
WS           float64
WS_per_48    float64
BPM          float64
VORP         float64
dtype: object

In [144]:
# scraping and cleaning data from drafts from 1966
# create a URL template that will allow us to access the web page for each year
url_template = "http://www.basketball-reference.com/draft/NBA_{year}.html"

# create empty dataframe
draft_df = pd.DataFrame()

In [145]:
# create loop to scrape and append our data to our other DataFrame
for year in range(1966, 2017):  # for each year
    url = url_template.format(year=year)  # get the url
    
    html = urlopen(url)  # get the html
    soup = BeautifulSoup(html, 'html5lib') # create our BS object
    

    # get our player data
    data_rows = soup.findAll('tr')[2:] 
    player_data = [[td.getText() for td in data_rows[i].findAll('td')]
                for i in range(len(data_rows))]
    
    # Turn yearly data into a DatFrame
    year_df = pd.DataFrame(player_data, columns=column_headers)
    # create and insert the Draft_Yr column
    year_df.insert(0, 'Draft_Yr', year)
    
    # Append to the big dataframe
    draft_df = draft_df.append(year_df, ignore_index=True)

In [146]:
draft_df.head()

,Draft_Yr,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,...,3P%,FT%,MP,PTS,TRB,AST,WS,WS/48,BPM,VORP
0,1966,1,NYK,Cazzie Russell,University of Michigan,12,817,22213,12377,3068,...,,.827,27.2,15.1,3.8,2.2,51.7,.112,-2.0,0.1
1,1966,2,DET,Dave Bing,Syracuse University,12,901,32769,18327,3420,...,,.775,36.4,20.3,3.8,6.0,68.8,.101,0.6,8.5
2,1966,3,SFW,Clyde Lee,Vanderbilt University,10,742,19885,5733,7626,...,,.614,26.8,7.7,10.3,1.1,33.5,.081,-2.4,-0.6
3,1966,4,STL,Lou Hudson,University of Minnesota,13,890,29794,17940,3926,...,,.797,33.5,20.2,4.4,2.7,81.0,.131,0.1,5.9
4,1966,5,BAL,Jack Marin,Duke University,11,849,24590,12541,4405,...,,.843,29.0,14.8,5.2,2.1,59.3,.116,-2.8,-1.4


In [147]:
draft_df.tail()

,Draft_Yr,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,...,3P%,FT%,MP,PTS,TRB,AST,WS,WS/48,BPM,VORP
6571,2016,56,DEN,Daniel Hamilton,University of Connecticut,1,5,26,12,5,...,.400,,5.2,2.4,1.0,1.4,0.1,.164,-1.0,0.0
6572,2016,57,MEM,Wang Zhelin,,,,,,,...,,,,,,,,,,
6573,2016,58,BOS,Abdel Nader,Iowa State University,1,35,349,106,49,...,.367,.720,10.0,3.0,1.4,0.6,0.1,.016,-4.9,-0.3
6574,2016,59,SAC,Isaiah Cousins,University of Oklahoma,,,,,,...,,,,,,,,,,
6575,2016,60,UTA,Tyrone Wallace,University of California,1,21,581,212,71,...,.308,.781,27.7,10.1,3.4,2.5,1.0,.085,-1.0,0.1


In [148]:
# cleaning draft_df

# convert data to proper data types
draft_df = draft_df.convert_objects(convert_numeric=True)

# get rid of the rows full of null values
draft_df = draft_df[draft_df.Player.notnull()]

# replace nulls with 0's
draft_df = draft_df.fillna(0)

# rename_columns
draft_df.rename(columns={'WS/48':'WS_per_48'}, inplace=True)
# Change % symbol
draft_df.columns = draft_df.columns.str.replace('%', '_Perc')
# Add per_G to per game stats
draft_df.columns.values[14:18] = [draft_df.columns.values[14:18][col] + 
                                  "_per_G" for col in range(4)]

# Changing the Data Types to int
draft_df.loc[:,'Yrs':'AST'] = draft_df.loc[:,'Yrs':'AST'].astype(int)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.


In [149]:
draft_df.dtypes

Draft_Yr       int64
Pk           float64
Tm            object
Player        object
College       object
Yrs            int32
G              int32
MP             int32
PTS            int32
TRB            int32
AST            int32
FG_Perc      float64
3P_Perc      float64
FT_Perc      float64
MP_per_G     float64
PTS_per_G    float64
TRB_per_G    float64
AST_per_G    float64
WS           float64
WS_per_48    float64
BPM          float64
VORP         float64
dtype: object

In [150]:
import numpy as np

draft_df['Pk'] = draft_df['Pk'].astype(np.int64) # change Pk to int data type

In [151]:
draft_df.dtypes

Draft_Yr       int64
Pk             int64
Tm            object
Player        object
College       object
Yrs            int32
G              int32
MP             int32
PTS            int32
TRB            int32
AST            int32
FG_Perc      float64
3P_Perc      float64
FT_Perc      float64
MP_per_G     float64
PTS_per_G    float64
TRB_per_G    float64
AST_per_G    float64
WS           float64
WS_per_48    float64
BPM          float64
VORP         float64
dtype: object

In [152]:
draft_df.isnull().sum() # test that thare are no missing values in our DataFrame

Draft_Yr     0
Pk           0
Tm           0
Player       0
College      0
Yrs          0
G            0
MP           0
PTS          0
TRB          0
AST          0
FG_Perc      0
3P_Perc      0
FT_Perc      0
MP_per_G     0
PTS_per_G    0
TRB_per_G    0
AST_per_G    0
WS           0
WS_per_48    0
BPM          0
VORP         0
dtype: int64

In [153]:
draft_df.head()
draft_df.tail()

,Draft_Yr,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,...,3P_Perc,FT_Perc,MP_per_G,PTS_per_G,TRB_per_G,AST_per_G,WS,WS_per_48,BPM,VORP
6571,2016,56,DEN,Daniel Hamilton,University of Connecticut,1,5,26,12,5,...,0.400,0.000,5.2,2.4,1.0,1.4,0.1,0.164,-1.0,0.0
6572,2016,57,MEM,Wang Zhelin,,0,0,0,0,0,...,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
6573,2016,58,BOS,Abdel Nader,Iowa State University,1,35,349,106,49,...,0.367,0.720,10.0,3.0,1.4,0.6,0.1,0.016,-4.9,-0.3
6574,2016,59,SAC,Isaiah Cousins,University of Oklahoma,0,0,0,0,0,...,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
6575,2016,60,UTA,Tyrone Wallace,University of California,1,21,581,212,71,...,0.308,0.781,27.7,10.1,3.4,2.5,1.0,0.085,-1.0,0.1


In [154]:
combnd_df = draft_df.append(df, ignore_index=True)

In [155]:
combnd_df = combnd_df[['Draft_Yr', 'Pk', 'Tm', 'Player', 'College', 'Yrs', 'G', 'MP', 'PTS', 'TRB', 'AST', 'FG_Perc', '3P_Perc',
                       'FT_Perc', 'MP_per_G', 'PTS_per_G', 'TRB_per_G', 'AST_per_G', 'WS', 'WS_per_48', 'BPM', 'VORP']]
combnd_df.tail()

,Draft_Yr,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,...,3P_Perc,FT_Perc,MP_per_G,PTS_per_G,TRB_per_G,AST_per_G,WS,WS_per_48,BPM,VORP
6043,2017,56,BOS,Jabari Bird,University of California,1,4,22,5,2,...,0.0,0.6,5.5,1.3,0.5,0.3,0.0,0.058,-4.6,0.0
6044,2017,57,BRK,Aleksandar Vezenkov,,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
6045,2017,58,NYK,Ognjen Jaramaz,,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
6046,2017,59,SAS,Jaron Blossomgame,Clemson University,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
6047,2017,60,ATL,Alpha Kaba,,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0


In [156]:
# save dataframe to csv
# draft_df.to_csv("data/draft_data_1966_to_2016.csv")
combnd_df.to_csv("data/draft_data_1966_to_2017.csv")

In [158]:
# download to local machine
from IPython.display import HTML
import base64

def create_download_link( df, title = "Download CSV file", filename = "draft_data_1966_to_2017.csv"):  
    csv = combnd_df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(df)